# Fourth Task: Prediction

We begin, as always, by importing all the necessary libraries and loading the dataset.

In [ ]:
#import warnings
#import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import scipy.stats as stats
#import seaborn as sns
#import unidecode

#from collections import defaultdict

#from scipy.stats import pearsonr, mode
#from scipy.spatial.distance import pdist, squareform
#from scipy.cluster.hierarchy import linkage, dendrogram

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import tree
import pydotplus 
from IPython.display import Image  
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
import wittgenstein as lw
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

#import os

cyclists = pd.read_csv('./dataset/cyclists_trasformed.csv')
races = pd.read_csv('./dataset/races_trasformed.csv')

Following the specification we consider only the races from 2022 onward and we consider only top-20 placement to defining the learning task as a binary classification task: one class indicating top placement, the other vice versa.

In [7]:
# Ensure the 'date' column is in datetime format
races['date'] = pd.to_datetime(races['date'])
races.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 589865 entries, 0 to 589864
Data columns (total 17 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   _url               589865 non-null  object        
 1   name               589865 non-null  object        
 2   points             589865 non-null  int64         
 3   length             589865 non-null  float64       
 4   climb_total        589865 non-null  int64         
 5   profile            589865 non-null  int64         
 6   startlist_quality  589865 non-null  int64         
 7   date               589865 non-null  datetime64[ns]
 8   position           589865 non-null  int64         
 9   cyclist            589865 non-null  object        
 10  cyclist_age        589865 non-null  int64         
 11  is_tarmac          589865 non-null  bool          
 12  cyclist_team       589865 non-null  object        
 13  delta              589865 non-null  float64 

# Data Preparation

Before employing the learning algorithms to fit a prediction model for the given purpose (= predict the final position in a race for a given cyclist), we need to perform a few adjustments to our data. 
Firstly, it is necessary to merge the two dataset since we need features from both tables in our learning algorithms.


In [ ]:
#TODO: DECIDERE DEI AVG POS E AVG DELTA DI CHI NON HA MAI GAREGGIATO 

# Merge con indicator=True
data_merged = pd.merge(races, cyclists, left_on='cyclist', right_on='_url', how='outer', indicator=True)

# Filtra le entries che non corrispondono
mismatched = data_merged[data_merged['_merge'] != 'both']

#print("\nMismatched entries:")
print(mismatched)

       _url_x name_x  points  length  climb_total  profile  startlist_quality  \
53046     NaN    NaN     NaN     NaN          NaN      NaN                NaN   
56942     NaN    NaN     NaN     NaN          NaN      NaN                NaN   
61189     NaN    NaN     NaN     NaN          NaN      NaN                NaN   
89300     NaN    NaN     NaN     NaN          NaN      NaN                NaN   
94337     NaN    NaN     NaN     NaN          NaN      NaN                NaN   
139563    NaN    NaN     NaN     NaN          NaN      NaN                NaN   
142896    NaN    NaN     NaN     NaN          NaN      NaN                NaN   
154121    NaN    NaN     NaN     NaN          NaN      NaN                NaN   
171916    NaN    NaN     NaN     NaN          NaN      NaN                NaN   
180790    NaN    NaN     NaN     NaN          NaN      NaN                NaN   
197846    NaN    NaN     NaN     NaN          NaN      NaN                NaN   
200906    NaN    NaN     NaN

Thereafter, we can reduce the size of our table by deleting the columns not useful for the purpose and adding one for specifying if a certain cyclist was in the first 20 position of a race or not. 

We deleted:

•*_url* (coming from cyclist) and *name* from (coming from races), since it is a repetition for *cyclist* in cyclist

• *weight* and *height* (coming from cyclist), since these characteristics are combined in the *bmi* feature we created 

• *nationality* (coming from cyclist), since it would be impractical to handle for its size (we left instead *continent*)

• *avg_position* and *avg_delta* (coming from cyclist), since it is better to consider more precise columns like *position* (see next) and *delta* from races

• *position* (coming from races) since we consider only the first 20 we will get from the new column 

• *birth_year* (coming from cyclist) because it's redundant having *cyclist_age* from races

• *cyclist_team* (coming from races) since we consider single cyclists

• *date* (coming from races) whose format is: "YYYY-MM-DD HH-MM-SS". From it, we can extract the information we need: we exclude the "HH-MM-SS" and the "MM-DD" part because we can easily group races based on the *season* of the year (attribute that we already have). At the end, we only care about the year for splitting the races for the training set.

We added:

• *top_20*, having value =1 if the corresponding cyclist was in between these positions or, on the opposite, =0.

In [ ]:
#Delete useless columns we cited
data_merged = data_merged.drop(columns=['_url_x', 'name_x', '_url_y', 'name_y', 'weight', 'height', 'birth_year', 'nationality', 'avg_position', 'avg_delta', 'cyclist_team'])
print(data_merged.head())

   points    length  climb_total  profile  startlist_quality  \
0   100.0  162000.0       1101.0      1.0             1241.0   
1   100.0   52000.0       1351.0      5.0             1241.0   
2    50.0   30000.0         -1.0     -1.0              388.0   
3   100.0  254000.0       3500.0      1.0             1362.0   
4   100.0  231000.0       4000.0      4.0             1362.0   

                 date  position           cyclist  cyclist_age is_tarmac  ...  \
0 1978-07-05 04:02:51      84.0  aad-van-den-hoek         27.0      True  ...   
1 1978-07-14 01:39:34      83.0  aad-van-den-hoek         27.0      True  ...   
2 1981-09-05 00:44:24       0.0  aad-van-den-hoek         30.0      True  ...   
3 1981-06-27 06:36:45     133.0  aad-van-den-hoek         30.0      True  ...   
4 1981-07-11 05:57:21      94.0  aad-van-den-hoek         30.0      True  ...   

   delta  month  season race_intensity            _url_y            name_y  \
0   27.0    7.0  summer       1.503613  aad-van-de

Now, we ensure data is in a suitable form for learning algorithms, in order to do so we have to normalize them and encode categorical data into numerical one. 
The begin with continous numeric attributes (*points*, *length*, *climb_total*, *startlist_quality*, *delta*, *race_intensity*, *bmi*) with the Z-score normalization, whose formula is z = (x - media) / standard_deviation.

In [ ]:
# Z-score normalization for points
data_merged['points'] = (data_merged['points'] - data_merged['points'].mean()) / data_merged['points'].std()

# Z-score normalization for length
data_merged['length'] = (data_merged['length'] - data_merged['length'].mean()) / data_merged['length'].std()

# Z-score normalization for climb_total
data_merged['climb_total'] = (data_merged['climb_total'] - data_merged['climb_total'].mean()) / data_merged['climb_total'].std()

# Z-score normalization for startlist_quality
data_merged['startlist_quality'] = (data_merged['startlist_quality'] - data_merged['startlist_quality'].mean()) / data_merged['startlist_quality'].std()

# Z-score normalization for delta
data_merged['delta'] = (data_merged['delta'] - data_merged['delta'].mean()) / data_merged['delta'].std()

# Z-score normalization for race_intensity
data_merged['race_intensity'] = (data_merged['race_intensity'] - data_merged['race_intensity'].mean()) / data_merged['race_intensity'].std()

# Z-score normalization for bmi
data_merged['bmi'] = (data_merged['bmi'] - data_merged['bmi'].mean()) / data_merged['bmi'].std()

Next, we normalize *profile*, *season*, *month* since their increasing values have a specific meaning. 

To do so, we employ **Min-Max Scaling Method**, which formula is x_scaled = (x - min(x)) / (max(x) - min(x)). 

In [ ]:
#Create instance for scaler
scaler = MinMaxScaler()

#Columns to which apply scaler
columns_to_scale = ['profile', 'month', 'season']

#Apply scaling and sobstitute values
data_merged[columns_to_scale] = scaler.fit_transform(data_merged[columns_to_scale])

Now we deal with the temporal attribute *date* and extrapolate only the year part. 

TWe posticipate its normalization since we need the raw information to divide our dataset for the train set; if we normalize now, it is difficult to derive the normalized value for '2022'.

In [ ]:
#Extrapolate only YY
data_merged['date'] = data_merged['date'].dt.year

To end, we need to transform categorical value into numerical ones. 

We mapped every *continent* to a number and transformed the boolean *is_tarmac* in 0/1 values. 
For what concerns *cyclist*, we use the **Target Encoding**: each cyclist is represented by a number reflecting his average probability of a top 20 placement (performance-based encoding).


In [ ]:
#Continent
#Extract unique 'continent'
unique_continents = data_merged['continent'].unique()

#Create mapping for each
continent_map = {continent: idx for idx, continent in enumerate(unique_continents, start=1)}

#Sobstitute values in the column
data_merged['continent'] = data_merged['continent'].map(continent_map)

#Is_tarmac
#Transform boolean in 0/1 value
data_merged['is_tarmac'] = data_merged['is_tarmac'].astype(int)

#Cyclist
#Calculate the average of the binary values ‘top_20’ for each cyclist
cyclist_mean_top20 = data_merged.groupby('cyclist')['top_20'].mean().to_dict()

#Replace in the ‘cyclist’ column in the dataset using the dictionary obtained
data_merged['cyclist'] = data_merged['cyclist'].map(cyclist_mean_top20)

To proceed, we divide the dataset in two parts: the races before 2022 (for training/validation set) and the races after 2022 (for the test set) and accurately define the binary classification: 1 for top-20 placement, 0 otherwise. 

In [ ]:
#Filter the dataset into training/validation and test sets
train_val_set = races[races['date'] < pd.Timestamp('2022')].copy() #Training/Validation Set
test_set = races[races['date'] >= pd.Timestamp('2022')].copy() #Test set

#Define the binary classification label: 1 for top-20 placement, 0 otherwise
train_val_set['top_20'] = (train_val_set['position'] <= 20).astype(int)
test_set['top_20'] = (test_set['position'] <= 20).astype(int)

#Print the resulting datasets for verification
print("Training/Validation set:")
print(train_val_set.info())

Training/Validation set:
<class 'pandas.core.frame.DataFrame'>
Index: 554459 entries, 0 to 589864
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   name               554459 non-null  object 
 1   points             554459 non-null  int64  
 2   length             554459 non-null  float64
 3   climb_total        554459 non-null  int64  
 4   profile            554459 non-null  int64  
 5   startlist_quality  554459 non-null  int64  
 6   cyclist_age        554459 non-null  int64  
 7   is_tarmac          554459 non-null  bool   
 8   delta              554459 non-null  float64
 9   month              554459 non-null  int64  
 10  season             554459 non-null  object 
 11  race_intensity     554459 non-null  float64
 12  top_20             554459 non-null  int64  
dtypes: bool(1), float64(3), int64(7), object(2)
memory usage: 55.5+ MB
None


In [10]:
print("\nTest set:")
print(test_set.head())


Test set:
               name  points    length  climb_total  profile  \
545  Tour de France     100  192900.0         3743        3   
546  Tour de France     100  192900.0         3743        3   
547  Tour de France     100  192900.0         3743        3   
548  Tour de France     100  192900.0         3743        3   
549  Tour de France     100  192900.0         3743        3   

     startlist_quality  cyclist_age  is_tarmac  delta  month  season  \
545               1551           30       True    0.0      7  summer   
546               1551           35       True   22.0      7  summer   
547               1551           30       True   26.0      7  summer   
548               1551           32       True   40.0      7  summer   
549               1551           24       True   49.0      7  summer   

     race_intensity  top_20  
545        2.128635       1  
546        2.128635       1  
547        2.128635       1  
548        2.128635       1  
549        2.128635       1

We conclude this section with *date* normalization, as previously promised.

In [ ]:
# Normalization for 'date'
data_merged['date'] = (data_merged['date'] - data_merged['date'].min()) / (data_merged['date'].max() - data_merged['date'].min())


# Learning Algorithms

The models we choose can be divided in different categories:

1) Tree-Based Models (Decision Tree, Random Forest, Gradient Boosting)

2) Support Vector Machines (SVM)

3) Naïve Bayes

4) K-Nearest Neighbors (KNN)

5) Neural Network

6) Rule-based

## Tree-Based Models (Decision Tree, Random Forest, Gradient Boosting)


### Decision Tree Model

Before computing the DT, we create a validation set to evaluate the model during training without using the test set; *train_val_set* contains only the (independent) features, while *train_val_label* contains the target binary values (=*top_20*).

The following code, divides the train_val_set into two sets:
- *train_set*: For training the model.
- *val_set*: To validate the model during tuning.

It also divides the train_val_label into:
- *train_label*: Target for training.
- *val_label*: Target for validation.

In [ ]:
#Separate the target (top_20) from the dataset
train_val_label = train_val_set.pop('top_20') #Contains the top_20 values for the training/validation set.
test_label = test_set.pop('top_20') #Contains the top_20 values for the test set.

#Split train_val_set into training and validation set
train_set, val_set, train_label, val_label = train_test_split(
    train_val_set, 
    train_val_label, 
    stratify=train_val_label, 
    test_size=0.30, #30% of the original training set is allocated for validation; 70% for training
    random_state=42
)

Model creation and training is obtained:

In [ ]:
#Creating and configuring the Decision Tree
dt = tree.DecisionTreeClassifier(
    criterion='gini',               #Use the Gini index to evaluate the purity of splits
    splitter='best',                #Splits the nodes by choosing the best split
    max_depth=10,                   #Limit tree depth to 10 levels
    min_samples_split=3,            #A node must have at least 3 samples to be split
    min_samples_leaf=4,             #Each leaf must contain at least 4 examples
    random_state=42                 #Ensures repeatability of results
)

#Train the model
dt = dt.fit(train_set, train_label)

ValueError: could not convert string to float: 'Volta a Catalunya'

Visualizing the actual Decision Tree obtained: 

In [ ]:
#To visualize the Decision Tree, you must intsall GraphViz
#MacOs: brew install graphviz
#Linux: sudo apt-get install graphviz
#Windows: Install from here https://graphviz.org/download/ 
#and add the following enviroment variable (the path can change)
#import os
#os.environ["PATH"] += os.pathsep + 'C:\Program Files (x86)\Graphviz2.38/bin/'

In [ ]:
dot_data = tree.export_graphviz(dt, out_file=None, 
                                feature_names=list(train_set.columns),  
                                class_names=['Non-Top 20', 'Top 20'],  
                                filled=True, rounded=True)  
graph = pydotplus.graph_from_dot_data(dot_data)  
Image(graph.create_png())


Evaluation of the Decision Tree is done employing predictions, then we compute:

• Accuracy (for both train and test set; percentage of correct predictions)

• Precision (Percentage of correct predictions among all positive predictions)

• Recall (Percentage of true positives identified)

• F1 Score (Harmonic mean of precision and recall.)

In [ ]:
#Predictions: use the trained model to make predictions on the training and test datasets.
train_pred_dt = dt.predict(train_set)
test_pred_dt = dt.predict(test_set)

#Evaluate metrics
from sklearn import metrics
print('Accuracy train set:', metrics.accuracy_score(train_label, train_pred_dt)) #Accuracy
print('Accuracy test set:', metrics.accuracy_score(test_label, test_pred_dt)) #Accuracy
print('Precision:', metrics.precision_score(test_label, test_pred_dt, average='weighted')) #Precision: Percentage of correct predictions among all positive predictions.
print('Recall:', metrics.recall_score(test_label, test_pred_dt, average='weighted')) #Recall
print('F1 Score:', metrics.f1_score(test_label, test_pred_dt, average='weighted')) #F1 Score


In [ ]:
#Compute and visualize Confusion Matrix
cm = confusion_matrix(test_label, test_pred_dt)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=dt.classes_)
disp.plot()
plt.show()

In [ ]:
#Scatter plot
plt.scatter(test_set.iloc[:, 1].values, test_set.iloc[:, 3].values, c=test_label.values, s=25, cmap='YlGn')
plt.title("Scatter Plot of Test Set")
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.show()


### Random Forest Model

Il Random Forest funziona combinando le predizioni di più alberi decisionali (Ensemble learning) per migliorare la robustezza e la precisione del modello.
Perciò, impiega il Bagging (Bootstrap Aggregating), cioè l'utilizzo di sottoinsiemi casuali dei dati e delle feature per ogni albero, riducendo l'overfitting e l'Out-of-bag evaluation, una tecnica interna che valuta le prestazioni senza bisogno di un validation set separato.

Costruzione degli Alberi:

- Ogni albero è costruito usando un sottoinsieme casuale dei dati (con ripetizione, bootstrap).

- Ad ogni split, considera un sottoinsieme casuale delle feature.

Predizione:

- Per la classificazione, aggrega le previsioni (voto a maggioranza).

- Per la regressione, calcola la media delle predizioni degli alberi.

In [18]:
#Random Forest definition and training
rf = RandomForestClassifier(
    n_estimators=30, #Number of trees in the forest
    criterion='gini', #Measure to evaluate the purity of the split
    max_features=3, #Maximum number of features considered for each split
    max_depth=4, #Maximum depth of trees
    min_samples_split=2, #Minimum number of samples to split a node
    min_samples_leaf=8, #Minimum number of samples in a leaf
    bootstrap=True, #Use bootstrap to create random subsets
    random_state=42 #Ensures repeatability of results
)

#Model training
rf = rf.fit(train_set, train_label)

NameError: name 'RandomForestClassifier' is not defined

In [ ]:
#Predictions on the test set
test_pred_rf = rf.predict(test_set)

In [ ]:
#Compute the performance of the model
print('Accuracy:', metrics.accuracy_score(test_label, test_pred_rf))
print('Precision:', metrics.precision_score(test_label, test_pred_rf, average='weighted'))
print('Recall:', metrics.recall_score(test_label, test_pred_rf, average='weighted'))
print('F1 Score:', metrics.f1_score(test_label, test_pred_rf, average='weighted'))

### Gradient Boost Model

Gradient Boosting is an ensemble technique that combines several simple models, usually decision trees, to create a more powerful model. It is based on the idea of building sequential models, where each model corrects the errors of the previous one.
How does it work?

- Iterative construction:
At each iteration, a new model is added that reduces the residual (error) of the previous model.
This is done by optimising an objective function (e.g. log-loss by classification).

- Model weighing:
Each successive model is weighted so that the hard errors of the previous model matter more.

- Learning Rate:
Controls the amount of correction that each additional model brings. A low learning rate increases accuracy, but requires more iterations.

In [ ]:
#Base model definition
base_model = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric="logloss")
# ypothetical hyperparameter space
hyperparameters = {
    "n_estimators": [25, 100, 250], #Number of trees in the forest
    "max_depth": [2, 3], #Maximum depth of trees
    "learning_rate": [1, 0.1, 0.001], #Update weight per iteration
    "subsample": [0.8, 1.0] #Percentage of data used in each tree
}

#Random search for tuning hyperparameters
search = RandomizedSearchCV(base_model, hyperparameters, cv=5, n_iter=10, random_state=42, scoring="f1_weighted")

#Training the model using the training set
search.fit(train_set, train_label)

#Best model found
clf = search.best_estimator_

#Training on the training set with the best model
clf.fit(train_set, train_label)

In [ ]:
#Prediction on test set
test_pred_xgb = clf.predict(test_set)

#Prediction of the model
print(classification_report(
    test_label, 
    test_pred_xgb, 
    target_names=['Non Top 20', 'Top 20']
))

### Final comparisons


In [ ]:
# Dizionario contenente i modelli allenati
models = {
    "Decision Tree": dt,           # Modello ad albero decisionale
    "Random Forest": rf,           # Modello Random Forest
    "Gradient Boosting": clf       # Modello Gradient Boosting
}

# Iterazione sui modelli per calcolare e stampare le metriche
for model_name, model in models.items():
    # Predizioni sul test set
    y_test_pred = model.predict(test_set)
    
    # Calcolo delle metriche
    acc = accuracy_score(test_label, y_test_pred)
    conf = confusion_matrix(test_label, y_test_pred)
    report = classification_report(test_label, y_test_pred, target_names=["Non Top 20", "Top 20"])

    # Output delle metriche
    print(f"\n--- {model_name} ---")
    print(f"Accuracy: {acc:.4f}")
    print(f"Confusion Matrix:\n{conf}")
    print(f"Classification Report:\n{report}")


 **Come leggere i risultati**

1. **`Accuracy`**:
   - Percentuale di predizioni corrette.
   - Utile per avere una visione generale delle performance, ma può essere ingannevole in caso di dataset sbilanciati.

2. **`Confusion Matrix`**:
   - Rappresenta visivamente i veri e falsi positivi/negativi.
   - Ogni cella indica:
     - **TN (0,0)**: Predizioni corrette per "Non Top 20".
     - **FP (0,1)**: Predizioni errate classificate come "Top 20".
     - **FN (1,0)**: Predizioni errate classificate come "Non Top 20".
     - **TP (1,1)**: Predizioni corrette per "Top 20".
   - Ti aiuta a identificare dove il modello sbaglia maggiormente.

3. **`Classification Report`**:
   - **Precision**:
     - Percentuale di predizioni corrette su tutte quelle fatte per una classe.
     - Formula: \( \text{Precision} = \frac{TP}{TP + FP} \).
     - Elevata precisione indica pochi falsi positivi.
   - **Recall (Sensibilità)**:
     - Percentuale di veri positivi catturati dal modello.
     - Formula: \( \text{Recall} = \frac{TP}{TP + FN} \).
     - Elevato recall indica pochi falsi negativi.
   - **F1-score**:
     - Media armonica di precision e recall, utile per bilanciare i due aspetti.
     - Formula: \( \text{F1-score} = 2 \cdot \frac{\text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}} \).
   - **Support**:
     - Numero di campioni effettivi per ogni classe.

---

 **Esempio di interpretazione**

Supponiamo che l'output del modello Gradient Boosting sia:

```
--- Gradient Boosting ---
Accuracy: 0.85
Confusion Matrix:
[[180  20]
 [ 15  85]]
Classification Report:
              precision    recall  f1-score   support
Non Top 20       0.92      0.90      0.91       200
    Top 20       0.81      0.85      0.83       100
```

- **Accuracy**: 85%, cioè l'85% delle predizioni sono corrette.
- **Confusion Matrix**:
  - **180 TN**: "Non Top 20" predetto correttamente.
  - **20 FP**: Predetto "Top 20", ma in realtà "Non Top 20".
  - **15 FN**: Predetto "Non Top 20", ma in realtà "Top 20".
  - **85 TP**: "Top 20" predetto correttamente.
- **Classification Report**:
  - Per la classe "Non Top 20":
    - Precision: 92% (pochi FP).
    - Recall: 90% (pochi FN).
  - Per la classe "Top 20":
    - Precision: 81% (più FP).
    - Recall: 85% (meno FN).

In questo caso, puoi confrontare le metriche tra i modelli per scegliere quello con il miglior bilanciamento di precision e recall per il tuo scopo.

## Support Vector Machines (SVM)

In [ ]:
#Creation and configuration of the SVM model
svm = SVC(
    kernel="sigmoid", #Kernel used to separate data
    C=0.5, #Penalisation for classification errors
    gamma="scale", #Influence of a single example on the model
    probability=True #Enable probability prediction
)

#Model training
svm.fit(train_set, train_label)

#Prediction of probabilities on test set
test_pred_proba_svm = svm.predict_proba(test_set)

#Prediction of classes on the test set
test_pred_svm = svm.predict(test_set)

In [ ]:
#Model performance output
print("Predizioni di probabilità:\n", test_pred_proba_svm[:5])  #First 5 probabilities
print("\nPredizioni delle classi:\n", test_pred_svm[:5])        #First 5 predictions

In [ ]:
#Calculation of performance metrics
print("\n--- SVM Classification Report ---")
print(classification_report(test_label, test_pred_svm, target_names=["Non Top 20", "Top 20"]))

In [ ]:
#Confusion matrix
conf_matrix = confusion_matrix(test_label, test_pred_svm)
print("\nConfusion Matrix:\n", conf_matrix)

## Naïve Bayes

In [ ]:
#Creating and training the Naïve Bayes model
gnb = GaussianNB()
gnb.fit(train_set, train_label)

#Prediction on test data
test_pred_gnb = gnb.predict(test_set)

In [ ]:
#Evaluation of model performance
print("\nNaïve Bayes Classifier")
print(classification_report(test_label, test_pred_gnb, target_names=["Non Top 20", "Top 20"]))

In [ ]:
#Confusion matrix
conf_matrix = confusion_matrix(test_label, test_pred_gnb)
print("\nConfusion Matrix:\n", conf_matrix)

## K-Nearest Neighbors (KNN)

n_neighbors=3: Considera i 3 vicini più prossimi per classificare un nuovo campione.
algorithm='ball_tree': Algoritmo che ottimizza la ricerca dei vicini più prossimi.
metric='minkowski': Misura la distanza Euclidea tra campioni.

In [ ]:
# Definizione del modello KNN con 3 vicini e algoritmo 'ball_tree'
knn = KNeighborsClassifier(n_neighbors=3, algorithm='ball_tree', metric='minkowski')

# Addestrare KNN sui dati di training
knn.fit(train_set, train_label)

# Prevedere sui dati di test
test_pred_knn = knn.predict(test_set)

# Valutare le performance del modello
print("\n K-Nearest Neighbors Classifier")
print(classification_report(test_label, test_pred_knn, target_names=["Non Top 20", "Top 20"]))


In [ ]:
#Confusion Matrix
conf_matrix = confusion_matrix(test_label, test_pred_knn)
print("\nConfusion Matrix:\n", conf_matrix)

## Neural Network

In [ ]:
# Convert labels to NumPy arrays
y_train = np.asarray(train_label).astype('float32').reshape((-1, 1))
y_test = np.asarray(test_label).astype('float32').reshape((-1, 1))

# Define the neural network model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(x_train.shape[1],)),
    tf.keras.layers.Dropout(0.2),  # Prevent overfitting
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # For binary classification
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_set, y_train,
    epochs=50,
    batch_size=64,
    validation_split=0.2,  # Use 20% of training data for validation
    verbose=1
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(train_set, y_test, verbose=0)
print(f"Test Accuracy: {test_accuracy:.4f}")


In [ ]:
# Plot the training and validation accuracy
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs, acc, 'bo-', label='Training Accuracy')
plt.plot(epochs, val_acc, 'ro-', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, loss, 'bo-', label='Training Loss')
plt.plot(epochs, val_loss, 'ro-', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Make predictions
test_pred_proba = model.predict(train_set)
test_pred = (test_pred_proba > 0.5).astype('int32')

from sklearn.metrics import classification_report
print(classification_report(y_test, test_pred, target_names=["Non Top 20", "Top 20"]))

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

cm = confusion_matrix(y_test, test_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Non Top 20", "Top 20"])
disp.plot(cmap='Blues')
plt.show()


## Rule based 

In [ ]:
ripper = lw.RIPPER()
param_grid = {"prune_size": [0.5, 0.6], "k": [1, 3, 5]}
grid_search = GridSearchCV(estimator=ripper, param_grid=param_grid, scoring='accuracy', cv=5)

grid_search.fit(train_set, train_label)

# Display the best configuration
print('Best parameters setting:', grid_search.best_params_)


In [ ]:
# Hyperparameter tuning for RIPPER
import wittgenstein as lw
from sklearn.model_selection import GridSearchCV

ripper = lw.RIPPER()
param_grid = {"prune_size": [0.5, 0.6], "k": [1, 3, 5]}
grid_search = GridSearchCV(estimator=ripper, param_grid=param_grid, scoring='accuracy', cv=5)

grid_search.fit(train_set, train_label)

# Display the best configuration
print('Best parameters setting:', grid_search.best_params_)


In [ ]:
# Predict on test set
test_pred = ripper.predict(test_set)

# Evaluate the model
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report

print("Accuracy:", accuracy_score(test_label, test_pred))
print("Precision:", precision_score(test_label, test_pred))
print("Recall:", recall_score(test_label, test_pred))
print("Classification Report:\n", classification_report(test_label, test_pred))


In [ ]:
#Scatter Plot for Predictions
plt.scatter(test_set.iloc[:, 0], test_set.iloc[:, 1], c=test_pred, cmap='RdBu', s=25)
plt.title('Rule-Based Predictions (Test Set)')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.colorbar(label='Predicted Class')
plt.show()


## Comparison between all models

- **ROC curve**: Displays the relationship between the True Positive Rate and the False Positive Rate. A curve closer to the upper left corner indicates better performance.

- **AUC (Area Under Curve)**: A higher AUC value indicates a better predictive ability of the model. The maximum value is 1 (perfect classifier), while 0.5 indicates a random model.

In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt

plt.figure(0).clf()

# Decision Tree
fpr, tpr, thresh = metrics.roc_curve(test_label, test_pred_dt)
auc = metrics.roc_auc_score(test_label, test_pred_dt)
plt.plot(fpr, tpr, label="DecisionTree, auc=" + str(round(auc, 3)))

# Naive Bayes
fpr, tpr, thresh = metrics.roc_curve(test_label, test_pred_gnb)
auc = metrics.roc_auc_score(test_label, test_pred_gnb)
plt.plot(fpr, tpr, label="Naive Bayes, auc=" + str(round(auc, 3)))

# XGBoost
fpr, tpr, thresh = metrics.roc_curve(test_label, test_pred_xgb)
auc = metrics.roc_auc_score(test_label, test_pred_xgb)
plt.plot(fpr, tpr, label="XGBoost Classifier, auc=" + str(round(auc, 3)))

# Random Forest
fpr, tpr, thresh = metrics.roc_curve(test_label, test_pred_rf)
auc = metrics.roc_auc_score(test_label, test_pred_rf)
plt.plot(fpr, tpr, label="RandomForest, auc=" + str(round(auc, 3)))

# K-Nearest Neighbor
fpr, tpr, thresh = metrics.roc_curve(test_label, test_pred_knn)
auc = metrics.roc_auc_score(test_label, test_pred_knn)
plt.plot(fpr, tpr, label="KNN, auc=" + str(round(auc, 3)))

# Support Vector Machine
fpr, tpr, thresh = metrics.roc_curve(test_label, test_pred_svm)
auc = metrics.roc_auc_score(test_label, test_pred_svm)
plt.plot(fpr, tpr, label="SVM, auc=" + str(round(auc, 3)))

# Neural Network
test_pred_nn = model.predict(test_set).ravel()  # Previsione delle probabilità per la rete neurale
fpr, tpr, thresh = metrics.roc_curve(test_label, test_pred_nn)
auc = metrics.roc_auc_score(test_label, test_pred_nn)
plt.plot(fpr, tpr, label="Neural Network, auc=" + str(round(auc, 3)))

# Rule-Based
test_pred_rb = ripper.predict_proba(test_set)[:, 1]  # Previsione delle probabilità con RIPPER
fpr, tpr, thresh = metrics.roc_curve(test_label, test_pred_rb)
auc = metrics.roc_auc_score(test_label, test_pred_rb)
plt.plot(fpr, tpr, label="Rule-Based, auc=" + str(round(auc, 3)))

# Layout
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Confronto delle Curve ROC')
plt.legend(loc=0)
plt.show()
